# Problema do Titanic do Kaggle
## ELT 574 - Aprendizado de Máquina
### Especialização em Inteligência Artificial e Computacional

#### Prof. Rodolpho Neves - NIAS/DEL/UFV

#### Problema:

O problema do desastre do **Titanic** é um clássico para quem está começando em **Machine Learning**. O objetivo é prever se um passageiro sobreviveu ou não ao naufrágio com base em algumas características. Dados baixados do Kaggle: [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic).

### Treinamento de um modelo de base (baseline model)

O treinamento do modelo de base seguirá as seguintes etapas:
1. Separação dos dados em treinamento e validação
2. Limpeza do banco de dados de treinamento e validação
3. Codificação das variáveis categóricas do problema
4. Ajuste de um modelo (RandomForest por simplicidade)
5. Comparação com os dados de validação 

Esperamos que este seja o primeiro modelo para que seja possível comparar estratégias de tratamento de dados. Para isso, vamos importar as bibliotecas a seguir.

#### 1. Separação dos dados em treinamento e validação

In [47]:
# manipulacao de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# aprendizado de maquina e selecao de modelo
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [48]:
# carregando os dados
dataset = pd.read_csv('kaggle-data/train.csv')
dataset.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [49]:
colunas_principais = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = dataset.loc[:, colunas_principais]
y = dataset.loc[:, 'Survived']

X.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S


#### 2. Limpeza do banco de dados de treinamento e validação

In [50]:
# separar o dataset em treino e validacao para ajustar o imputer
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train.shape, X_val.shape

((712, 7), (179, 7))

In [51]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 134 to 102
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       573 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


#### 3. Transformação das variáveis do problema

##### Pipeline de processamento

In [52]:
# separacao das variaveis
variaveis_principais = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
variaveis_numericas = ['SibSp', 'Parch', 'Age', 'Fare']
variaveis_categoricas = ['Pclass', 'Sex', 'Embarked']

# instanciacao dos pipelines de preprocessamento
pipeline_numericos = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), # imputer de media
    ('scaler', StandardScaler()) # padronizacao
])
pipeline_categoricos = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')), # imputer de moda
    ('encoder', OneHotEncoder()) # one-hot encoding
])

# transformacao das colunas
preprocessor = ColumnTransformer([
    ('numericos', pipeline_numericos, variaveis_numericas),
    ('categoricos', pipeline_categoricos, variaveis_categoricas)
])

In [53]:
# # teste de funcionamento
# preprocessor.fit(X)
# X_pipeline = preprocessor.transform(X) # saida é um numpy array
# X_pipeline[0]

In [54]:
# # teste de funcionamento
# X_pipeline = pd.DataFrame(X_pipeline, columns=preprocessor.get_feature_names_out())
# X_pipeline.head(3)

#### 4. Ajuste de um modelo (RandomForest por simplicidade)

In [55]:
# pipeline do modelo
model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [56]:
# ajuste do modelo
model.fit(X_train, y_train)

# previsoes
y_pred = model.predict(X_val)

# acuracia
print(f'Acurácia: {accuracy_score(y_val, y_pred):.2f}')

Acurácia: 0.79


#### 5. Comparação com os dados de validação 

In [57]:
# Avaliação do modelo
y_pred = model.predict(X_val)
print(f'Acurácia: {accuracy_score(y_val, y_pred):.2f}')

Acurácia: 0.79


##### 5.1 Retreinar o modelo com todos os dados, sem separação


In [58]:
model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numericos',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SibSp', 'Parch', 'Age',
                                                   'Fare']),
                                                 ('categoricos',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['Pclass', 'Sex',
                                                   'Embarked'])])),
                ('model', RandomForestClassifier(random_state=42))])

#### 6. Criar arquivo para submissão na competição

In [59]:
# carregar os dados de teste
dataset_test = pd.read_csv('kaggle-data/test.csv')

# selecionar os dados de teste
colunas_principais = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_test = dataset_test.loc[:, colunas_principais]

# prever os resultados
y_pred = model.predict(X_test)

# salvar os resultados
dataset_test['Survived'] = y_pred
dataset_test[['PassengerId', 'Survived']].to_csv('kaggle-data/submission.csv', index=False)

#### 7. Salvar o modelo para app

In [60]:
import pickle

# salvar o modelo
with open('model-titanic-pipeline.pkl', 'wb') as file:
    pickle.dump(model, file)